## 2.3. 自动求梯度
&emsp;&emsp;在深度学习中，我们经常需要对函数求梯度(gradient)。本节将介绍如果使用tensorflow提供的tf.GradientTape API来自动求梯度。

In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

###2.3.1. 简单例子
&emsp;&emsp;我们先看一个简单的例子:$y=2x^Tx$求关于列向量$x$的梯度。我们先创建变量x,并赋初值。

In [6]:
x=tf.reshape(tf.range(4,dtype=tf.float32),shape=(4,1),)
x

<tf.Tensor: id=14, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

&emsp;&emsp;为了求有关变量x的梯度，我们需要

In [0]:
with tf.GradientTape() as t:
  t.watch(x)
  y=2*tf.matmul(x,x,transpose_a=True)

In [8]:
t.gradient(y,x).numpy()

array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)

### 2.3.2. 对Python控制流求梯度
&emsp;&emsp;使用tf动态计算图的一个便利之处是，即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。

&emsp;&emsp;考虑下面程序，其中包含Python的条件和循环控制。需要强调的是，这里循环（while循环）迭代的次数和条件判断（if语句）的执行都取决于输入a的值。

In [0]:
def f(a):
  b=a*2
  while tf.norm(b).numpy()<1000:
    b=b*2
  if tf.reduce_sum(b).numpy()>0:
    c=b
  else:
    c=100*b
  return c

In [19]:
a=tf.ones([1])
a.numpy(),f(a).numpy()

(array([1.], dtype=float32), array([1024.], dtype=float32))

&emsp;&emsp;像之前一样使用tf.GradientTape()记录并求梯度

In [0]:
with tf.GradientTape() as t:
  t.watch(a)
  c=f(a)
  

In [21]:
dc_da=t.gradient(c,a)
dc_da.numpy()

array([1024.], dtype=float32)

In [0]:
assert dc_da.numpy()==c.numpy()/a.numpy()

###2.3.3. 小结
* tensorflow提供tf.GradientTape()API来自动化求导过程，可以对一般的命令式程序进行求导。